# Benchmarking high performance pandas alternatives  

@Author: Zoumana KEITA - Senior Data Scientist at IFC and Expert Content Writer

## Setting up the Benchmarking Environment

### Creating the benchmarking data

In [1]:
import pandas as pd

data_URL = "https://raw.githubusercontent.com/keitazoumana/Experimentation-Data/main/diabetes.csv"

original_data = pd.read_csv(data_URL)

# Duplicated each row 100000 times
benchmarking_df = original_data.loc[original_data.index.repeat(100000)]

In [2]:
benchmarking_df.drop(['Outcome'], axis=1, inplace=True)
benchmarking_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
0,6,148,72,35,0,33.6,0.627,50
0,6,148,72,35,0,33.6,0.627,50
0,6,148,72,35,0,33.6,0.627,50
0,6,148,72,35,0,33.6,0.627,50


In [3]:
benchmarking_df.shape

(76800000, 8)

### Install relevent libraries

In [3]:
!pip3 install polars
!pip3 install vaex
!pip install "git+https://github.com/h2oai/datatable.git"

In [3]:
import polars as pl
import vaex as vx
import pandas as pd
import datatable as dt
import plotly.express as px

In [35]:
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)  

## Start the benchmarking

In [4]:
file_name = "benchmarking_data.csv"

In [5]:
# Save the final benchmarking data
benchmarking_df.to_csv(file_name, index=False)

In [6]:
from time import time

### Execution Time

#### 1. Data Loading

In [98]:
def read_csv_with_time(library_name, file_name):

    final_time = 0

    start_time = time()

    if library_name.lower() == 'polars':
        df = pl.read_csv(file_name)

    elif library_name.lower() == 'pandas':
        df = pd.read_csv(file_name)

    elif library_name.lower() == 'vaex':
        df = vx.read_csv(file_name)

    elif library_name.lower() == 'datatable':
        df = dt.fread(file_name)

    else:
        raise ValueError("Invalid library name. Must be 'polars', 'pandas', 'vaex', or 'datatable'")

    end_time = time()

    final_time = end_time - start_time

    return {"library": library_name, "execution_time": final_time}

In [99]:
pandas_time = read_csv_with_time('pandas', file_name)
polars_time = read_csv_with_time('polars', file_name)
vaex_time = read_csv_with_time('vaex', file_name)
datatable_time = read_csv_with_time('datatable', file_name)

In [100]:
exec_times = [pandas_time, polars_time,
                vaex_time, datatable_time]

In [101]:
exec_times

[{'library': 'pandas', 'execution_time': 57.92357087135315},
 {'library': 'polars', 'execution_time': 12.994961261749268},
 {'library': 'vaex', 'execution_time': 69.38114285469055},
 {'library': 'datatable', 'execution_time': 6.504034996032715}]

In [102]:
def plot_metrics(list_exec_time, graph_title):

    df = pd.DataFrame(list_exec_time)

    fig = px.bar(df, x='library', y= 'execution_time', title=graph_title)
    
    fig.show()

In [103]:
plot_metrics(exec_times,
             graph_title="Data Loading Time Comparison")

In [ ]:
for dic in exec_times:
    print(dic)

#### 2. Data Grouping

In [24]:
def group_data_with_time(library_name, df, column_name='Pregnancies'):

    start_time = time()

    if library_name.lower() == 'polars':
        df_grouped = df.groupby(column_name).first()

    elif library_name.lower() == 'vaex':
        df_grouped = df.groupby(column_name)

    elif library_name.lower() == 'pandas':
        df_grouped = df.groupby(column_name)

    elif library_name.lower() == 'datatable':
        df_grouped = df[:, :, dt.by(column_name)]
    else:
        raise ValueError("Invalid library name. Must be 'polars', 'vaex', or 'datatable'")

    end_time = time()

    final_time = end_time - start_time

    return {"library": library_name, "execution_time": final_time}

In [21]:
pandas_df = pd.read_csv(file_name)
polars_df = pl.read_csv(file_name)
vaex_df = vx.read_csv(file_name)
datatable_df = dt.fread(file_name)

In [25]:
pandas_time = group_data_with_time('pandas', pandas_df)
polars_time = group_data_with_time('polars', polars_df)
vaex_time = group_data_with_time('vaex', vaex_df)
datatable_time = group_data_with_time('datatable', datatable_df)

In [37]:
exec_times = [pandas_time, polars_time,
                vaex_time, datatable_time]

In [39]:
exec_times

[{'library': 'pandas', 'execution_time': 0.008199930191040039},
 {'library': 'polars', 'execution_time': 7.193098068237305},
 {'library': 'vaex', 'execution_time': 1.270841121673584},
 {'library': 'datatable', 'execution_time': 0.8126001358032227}]

In [41]:
plot_metrics(exec_times,
             graph_title="Data Grouping Time Comparison")

#### 3. Column Sorting

In [43]:
def sort_data_with_time(library_name, df, column_name='Pregnancies'):
    
    start_time = time()

    if library_name.lower() == 'polars':
        df_sorted = df.sort(column_name)
    elif library_name.lower() == 'vaex':
        df_sorted = df.sort(column_name)

    elif library_name.lower() == 'datatable':
        df_sorted = df.sort(column_name)
        
    elif library_name.lower() == 'pandas':
        df_sorted = pd.DataFrame(df).sort_values(column_name)
    else:
        raise ValueError("Invalid library name. Must be 'polars', 'vaex', 'datatable', or 'pandas'")

    end_time = time()

    final_time = end_time - start_time

    return {"library": library_name, "execution_time": final_time}


In [44]:
pandas_time = sort_data_with_time('pandas', pandas_df)
polars_time = sort_data_with_time('polars', polars_df)
vaex_time = sort_data_with_time('vaex', vaex_df)
datatable_time = group_data_with_time('datatable', datatable_df)

In [45]:
exec_times = [pandas_time, polars_time,
                vaex_time, datatable_time]

In [47]:
exec_times

[{'library': 'pandas', 'execution_time': 21.7365460395813},
 {'library': 'polars', 'execution_time': 13.50932002067566},
 {'library': 'vaex', 'execution_time': 2.543069839477539},
 {'library': 'datatable', 'execution_time': 0.9501991271972656}]

In [46]:
plot_metrics(exec_times,
             graph_title="Column Sorting Time Comparison")

#### 4. Data Offloading

In [50]:
def offload_data_with_time(library_name, df):
    
    start_time = time()

    if library_name.lower() == 'polars':
        array = df.to_numpy()
        
    elif library_name.lower() == 'vaex':
        array = df.to_pandas_df().values

    elif library_name.lower() == 'datatable':
        array = df.to_numpy()
        
    elif library_name.lower() == 'pandas':
        array = pd.DataFrame(df).values
    else:
        raise ValueError("Invalid library name. Must be 'polars', 'vaex', 'datatable', or 'pandas'")

    end_time = time()

    final_time = end_time - start_time

    return {"library": library_name, "execution_time": final_time}


In [51]:
pandas_time = offload_data_with_time('pandas', pandas_df)
polars_time = offload_data_with_time('polars', polars_df)
vaex_time = offload_data_with_time('vaex', vaex_df)
datatable_time = offload_data_with_time('datatable', datatable_df)

In [52]:
exec_times = [pandas_time, polars_time,
                vaex_time, datatable_time]

In [53]:
exec_times

[{'library': 'pandas', 'execution_time': 12.863996267318726},
 {'library': 'polars', 'execution_time': 8.75653600692749},
 {'library': 'vaex', 'execution_time': 33.69999098777771},
 {'library': 'datatable', 'execution_time': 4.424616813659668}]

In [54]:
plot_metrics(exec_times,
             graph_title="Data Offloading Time Comparison")

### Bencharking Memory usage

In [75]:
import tracemalloc as tm
import os

In [84]:
list_memory_usage = []

In [85]:
# starting the monitoring
tm.start()
offload_data_with_time('pandas', pandas_df)

# Get the memory usage
memory_usage = tm.get_traced_memory()

tm.stop()

list_memory_usage.append({
    'library': 'pandas',
    'memory_usage': memory_usage
})

In [86]:
tm.start()
polars_time = offload_data_with_time('polars', polars_df)
memory_usage = tm.get_traced_memory()

tm.stop()

list_memory_usage.append({
    'library': 'polars',
    'memory_usage': memory_usage
})

In [87]:
tm.start()
vaex_time = offload_data_with_time('vaex', vaex_df)
memory_usage = tm.get_traced_memory()

tm.stop()

list_memory_usage.append({
    'library': 'vaex',
    'memory_usage': memory_usage
})

In [88]:
tm.start()
datatable_time = offload_data_with_time('datatable', datatable_df)
memory_usage = tm.get_traced_memory()

tm.stop()

list_memory_usage.append({
    'library': 'datatable',
    'memory_usage': memory_usage
})

In [96]:
def plot_memory_usage(list_memory_usage, graph_title='Memory Usage by Library'):

    df = pd.DataFrame(list_memory_usage)

    # separate the memory usage tuple into two columns: current_memory and peak_memory
    df[['current_memory', 'peak_memory']] = pd.DataFrame(df['memory_usage'].tolist(), index=df.index)

    # now we no longer need the memory_usage column
    df = df.drop(columns='memory_usage')

    # melt the DataFrame to make it suitable for grouped bar chart
    df_melted = df.melt(id_vars='library', var_name='memory_type', value_name='memory')

    # create the grouped bar chart
    fig = px.bar(df_melted, x='library', y='memory', color='memory_type', barmode='group',
                 labels={'memory':'Memory Usage (bytes)', 'library':'Library', 'memory_type':'Memory Type'},
                 title=graph_title)
    
    fig.update_layout(yaxis_type="log")
    fig.show()


In [94]:
list_memory_usage

[{'library': 'pandas', 'memory_usage': (2453156, 12289511750)},
 {'library': 'polars', 'memory_usage': (2232, 12792)},
 {'library': 'vaex', 'memory_usage': (332133, 12289341330)},
 {'library': 'datatable', 'memory_usage': (83317, 93877)}]

In [97]:
plot_memory_usage(list_memory_usage)

### Bencharking Data grouping